In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
import warnings

In [ ]:
with warnings.catch_warnings():
        warnings.simplefilter("ignore")

In [ ]:
def realizar_eda(dataframe):
    # Colores personalizados para las gráficas
    colores = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
               '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

    # Ignorar advertencias
    warnings.filterwarnings("ignore")

    # Información general del DataFrame
    print("Información general del DataFrame:")
    print(dataframe.info())

    # Estadísticas descriptivas
    print("\nEstadísticas descriptivas:")
    print(dataframe.describe())

    # Completitud (porcentaje de valores no nulos)
    print("\nCompletitud:")
    completitud = (1 - dataframe.isnull().mean()) * 100
    print(completitud.round(2))

    # Histogramas para columnas numéricas con valores numéricos
    numericas = dataframe.select_dtypes(include=['int64', 'float64'])
    for columna in numericas.columns:
        plt.figure(figsize=(8, 4))
        ax = sns.histplot(data=dataframe, x=columna, kde=True, color=colores[0])
        plt.title(f'Histograma de {columna}')
        if len(ax.patches) < 25:  # Imprimir valores solo si hay menos de 25 barras
            for p in ax.patches:
                ax.annotate(f'{p.get_height():.0f}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom')
        plt.show()

    # Gráfico de barras para columnas categóricas
    categoricas = dataframe.select_dtypes(include=['object'])
    for columna in categoricas.columns:
        plt.figure(figsize=(8, 4))
        ax = sns.countplot(data=dataframe, x=columna, palette=colores)
        plt.title(f'Gráfico de barras de {columna}')
        plt.xticks(rotation=90)
        if len(ax.patches) < 25:  # Imprimir valores solo si hay menos de 25 barras
            for p in ax.patches:
                ax.annotate(f'{p.get_height():.0f}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom')
        plt.show()

    # Matriz de correlación
    print("\nMatriz de Correlación:")
    matriz_correlacion = dataframe.corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(matriz_correlacion, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
    plt.title("Matriz de Correlación")
    plt.show()

    # Boxplots para columnas numéricas en orientación horizontal
    for columna in numericas.columns:
        plt.figure(figsize=(8, 4))
        ax = sns.boxplot(data=dataframe, x=columna, orient='h', color=colores[1])
        plt.title(f'Boxplot de {columna}')
        plt.show()

    # Matriz de dispersión para explorar multicolinealidad visualmente
    print("\nMatriz de Dispersión para Multicolinealidad Visual:")
    sns.pairplot(dataframe[numericas.columns], palette=colores)
    plt.show()

In [ ]:
# Función para llenar valores faltantes o eliminar columnas
def llenar_o_eliminar_valores(df, completitud_minima, metodo_por_columna):
    for columna in metodo_por_columna.keys():
        completitud = df[columna].count() / len(df)  # Calcular la completitud
        if completitud > completitud_minima:
            metodo = metodo_por_columna.get(columna, None)  # Obtener el método del diccionario
            if metodo:
                if metodo == 'mean':
                    df[columna].fillna(df[columna].mean(), inplace=True)  # Llenar con la media
                elif metodo == 'median':
                    df[columna].fillna(df[columna].median(), inplace=True)  # Llenar con la mediana
                elif metodo == 'mode':
                    df[columna].fillna(df[columna].mode()[0], inplace=True)  # Llenar con la moda
                elif metodo == 'ffill':
                    df[columna].fillna(method='ffill', inplace=True)  # Llenar con método ffill (adelante)
                elif metodo == 'bfill':
                    df[columna].fillna(method='bfill', inplace=True)  # Llenar con método bfill (atrás)
        else:
            print(f"Eliminando columna '{columna}' debido a baja completitud")
            df.drop(columna, axis=1, inplace=True)  # Eliminar la columna si la completitud es baja
    return df

In [ ]:
# Función para ajustar valores atípicos según el método elegido
def manejar_outliers_por_metodo(df, metodo='rango', atipicos_threshold=1.5):
    df_manipulado = df.copy()  # Crear una copia del DataFrame para no modificar el original

    for columna in df_manipulado.columns:
        if df_manipulado[columna].dtype != 'object':
            q1 = df_manipulado[columna].quantile(0.25)
            q3 = df_manipulado[columna].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - atipicos_threshold * iqr # q1 - 2*df[columna].std()
            upper_bound = q3 + atipicos_threshold * iqr # q1 + 2*df[columna].std()

            atipicos_indices = (df_manipulado[columna] < lower_bound) | (df_manipulado[columna] > upper_bound)

            if metodo == 'rango':
                df_manipulado[columna] = df_manipulado[columna].clip(lower_bound, upper_bound)  # Ajustar al rango mínimo-máximo
            elif metodo == 'media':
                df_manipulado[columna] = df_manipulado[columna].mask(atipicos_indices, df_manipulado[columna].mean())  # Reemplazar por la media
            elif metodo == 'mediana':
                df_manipulado[columna] = df_manipulado[columna].mask(atipicos_indices, df_manipulado[columna].median())  # Reemplazar por la mediana

    return df_manipulado

In [ ]:
def Escalar(df, columnas, metodo):
  if metodo == "Estandar":
    scaler = StandardScaler()
  elif metodo == "Minmax":
    scaler =MinMaxScaler()
  df[columnas] = scaler.fit_transform(df[columnas])

  return df

In [ ]:
def Dummies(df,columnas):
  return pd.get_dummies(df, columns=columnas, drop_first=False)

In [ ]:
def Labels(df,columnas):
  label_encoder = LabelEncoder()
  df[columnas] = label_encoder.fit_transform(df[columnas])
  return df